In [1]:
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
import sys
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

# 初始化模型参数

In [2]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))
    
    W_xz, W_hz, b_z = _three()  # 更新门参数
    W_xr, W_hr, b_r = _three()  # 重置门参数
    W_xh, W_hh, b_h = _three()  # 候选隐藏状态参数
    
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q])

will use cuda


# 定义模型

In [3]:
def init_gru_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

In [4]:
#下面根据门控循环单元的计算表达式定义模型。
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        Z = torch.sigmoid(torch.matmul(X, W_xz) + torch.matmul(H, W_hz) + b_z)
        R = torch.sigmoid(torch.matmul(X, W_xr) + torch.matmul(H, W_hr) + b_r)
        H_tilda = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(R * H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

# 训练模型并创作歌词

In [10]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 500, 20, 100, 1e2, 1e-2
pred_period, pred_len, prefixes = 10, 10, ['雪地', '雪地']

In [11]:
d2l.train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)

epoch 10, perplexity 76.791065, time 0.17 sec
 - 雪地          
 - 雪地          
epoch 20, perplexity 41.322997, time 0.22 sec
 - 雪地   许的的的   
 - 雪地   许的的的   
epoch 30, perplexity 22.085654, time 0.21 sec
 - 雪地    许许的了就该
 - 雪地    许许的了就该
epoch 40, perplexity 15.625770, time 0.20 sec
 - 雪地    许许的了 就
 - 雪地    许许的了 就
epoch 50, perplexity 9.737393, time 0.21 sec
 - 雪地    许我的 就应
 - 雪地    许我的 就应
epoch 60, perplexity 6.540660, time 0.18 sec
 - 雪地里来 男：你我们 住
 - 雪地里来 男：你我们 住
epoch 70, perplexity 4.708066, time 0.22 sec
 - 雪地里不出来 男：我我们
 - 雪地里不出来 男：我我们
epoch 80, perplexity 3.256253, time 0.25 sec
 - 雪地里不会—a    谁
 - 雪地里不会—a    谁
epoch 90, perplexity 2.450976, time 0.26 sec
 - 雪地里不出来 男：我离们
 - 雪地里不出来 男：我离们
epoch 100, perplexity 1.872841, time 0.26 sec
 - 雪地里相爱 他们说零下已
 - 雪地里相爱 他们说零下已
epoch 110, perplexity 1.667968, time 0.26 sec
 - 雪地里相爱 他们说零下已
 - 雪地里相爱 他们说零下已
epoch 120, perplexity 1.560952, time 0.31 sec
 - 雪地里相爱 他们说零下已
 - 雪地里相爱 他们说零下已
epoch 130, perplexity 1.245422, time 0.25 sec
 - 雪地里相爱 他们说零下已
 - 雪地里相

# 简洁实现

In [12]:
lr = 1e-2 # 注意调整学习率
gru_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens)
model = d2l.RNNModel(gru_layer, vocab_size).to(device)
d2l.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 10, perplexity 3.555386, time 0.04 sec
 - 雪地里相爱 女：你依旧 
 - 雪地里相爱 女：你依旧 
epoch 20, perplexity 1.070259, time 0.03 sec
 - 雪地里相爱 他们说零下已
 - 雪地里相爱 他们说零下已
epoch 30, perplexity 1.017841, time 0.03 sec
 - 雪地里相爱 他们说零下已
 - 雪地里相爱 他们说零下已
epoch 40, perplexity 1.053392, time 0.04 sec
 - 雪地里相爱 他们说零下已
 - 雪地里相爱 他们说零下已
epoch 50, perplexity 1.013979, time 0.05 sec
 - 雪地里相爱 他们说零下已
 - 雪地里相爱 他们说零下已
epoch 60, perplexity 1.012569, time 0.04 sec
 - 雪地里相爱 他们说零下已
 - 雪地里相爱 他们说零下已
epoch 70, perplexity 1.013592, time 0.05 sec
 - 雪地里相爱 他们说零下已
 - 雪地里相爱 他们说零下已
epoch 80, perplexity 1.011953, time 0.05 sec
 - 雪地里相爱 他们说零下已
 - 雪地里相爱 他们说零下已
epoch 90, perplexity 1.014935, time 0.05 sec
 - 雪地里相爱 他们说零下已
 - 雪地里相爱 他们说零下已
epoch 100, perplexity 1.013765, time 0.05 sec
 - 雪地里相爱 他们说零下已
 - 雪地里相爱 他们说零下已
epoch 110, perplexity 1.016119, time 0.04 sec
 - 雪地里相爱 他们说零下已
 - 雪地里相爱 他们说零下已
epoch 120, perplexity 1.010307, time 0.05 sec
 - 雪地里相爱 他们说零下已
 - 雪地里相爱 他们说零下已
epoch 130, perplexity 1.013107, time 0.04 sec
 - 雪地里相爱 他们说零下已
 - 雪地里相爱 他们